# Example: The Clifford Torus

We denote $S^3 = \{x\in\mathbb{R}^4\vert \Vert x\Vert = 1\}$ for the $3$-dimensional unit sphere.
The [Clifford torus](https://en.wikipedia.org/wiki/Clifford_torus) is defined by
\begin{align}
T = \left\{\frac1{\sqrt 2}\left(\cos\theta,\sin\theta,\cos\varphi,\sin\varphi\right)\mid 0\leq\theta\leq2\pi,0\leq\varphi\leq2\pi\right\}\subset\mathbb R^4
\end{align}
One can easily show that $T$ is a minimal surface in $S^3$.

Since $T$ is a submanifold of $S^3$ we may project it into $\mathbb R^3$ using the stereographic projection
\begin{align}
S^3\setminus\{(0,0,0,1)\}\ni (x, y, z, w)\mapsto \left(\frac{x}{1-w},\frac{y}{1-w},\frac{z}{1-w}\right)\in\mathbb R^3
\end{align}
Using ```maniflow``` we can implement the stereographic projection as a ```VertexFunction```:

In [1]:
from maniflow.mesh.utils import VertexFunction

@VertexFunction
def stereographicProjection(vertex):
    return vertex[:3:] / (1 - vertex[-1])

Since $T\subset\mathbb R^4$ we can rotate the Clifford torus around the $w$-axis by the angle $\alpha$ by applying the following rotation matrix:
\begin{align}
\begin{pmatrix}1&0&0&0\\0&\cos\alpha&0&\sin\alpha\\0&0&1&0\\0&-\sin\alpha&0&\cos\alpha\end{pmatrix}
\end{align}

The goal of this example will be to let the Clifford torus $T$ perform a simple rotation in the $w$-axis. By projecting it into $\mathbb R^3$ by the stereographic projection we can then render it.

## Capturing the Clifford torus in code

By the definition above we may model the Clifford torus in ```maniflow``` by mapping $[0,2\pi]\times[0,2\pi]$ to the Clifford torus using the mapping
\begin{align}
(\theta, \varphi)\mapsto\frac1{\sqrt 2}\left(\cos\theta,\sin\theta,\cos\varphi,\sin\varphi\right)
\end{align}
Using ```maniflow``` we can model this by the following

In [2]:
import numpy as np
from maniflow.mesh.parameterized import Grid

grid = Grid((0, 2*np.pi), (0, 2*np.pi), 40, 40)  # creating a discrete version of the set [0,2pi]x[0,2pi]

# modeling the parametrization as a VertexFunction
@VertexFunction
def clifford_torus(vertex):
    theta, phi = vertex[0], vertex[1]
    return np.array([np.cos(theta), np.sin(theta), np.cos(phi), np.sin(phi)]) / np.sqrt(2)

# creating the Mesh object that is the Clifford torus
torus = clifford_torus(grid)

## Creating a method for rotating the Clifford torus

Now we have obtained the mesh ```torus```. The next step will be to create a method for rotating the Clifford torus. By using the considerations from above we may use the rotation matrix. Let ```N``` be the number of frames our animation will have:

In [3]:
N = 300  # we arbitrarily set the number of frames to 300

# create a function that will accept an angle and return the desired rotation matrix as described above
rotationMatrix = lambda alpha: np.array([
    [1, 0, 0, 0],
    [0, np.cos(alpha), 0, np.sin(alpha)],
    [0, 0, 1, 0],
    [0, -np.sin(alpha), 0, np.cos(alpha)]
])


# we can now create another VertexFunction that will apply this rotation matrix to every vertex 
@VertexFunction
def rotate(vertex):
    return rotationMatrix(2 * np.pi / N).dot(vertex)  # we want to rotate by 2pi/N to perform one full rotation in the animation

## Creating the animation

Now we can turn our attention to the animation. The procedure is very simple: we rotate the Clifford torus in each frame by the angle $\frac{2\pi}N$ and project it with the stereographic projection into the R^3. Then we can render the frame.

In [ ]:
from maniflow.render import SVGPainterRenderer
from maniflow.render.scene import *


camera = Camera(position=(25., 2., 5.), target=(0., 0., 0.))
scene = Scene(camera=camera, width=720, height=720, light=np.array([20., 0., 0.]))
renderer = SVGPainterRenderer(scene)
shader = Shader(fill=[230., 232., 252.], fill_opacity=60, stroke=[25., 193., 209., 255.],
                stroke_width="0.6", specular=20, shininess=60,
                back_fill=[230., 232., 252.], back_opacity=30.,
                back_stroke=[25., 193., 209., 255.])

for i in range(N):
    projected = stereographicProjection(torus)
    projected.setShader(shader)
    image = renderer.render(projected)
    image.save_svg("render/clifford" + str(i) + ".svg")
    torus = rotate(torus)

Now we can bring the individual frames together to form a GIF. For this we use ```ffmpeg```:
```bash
ffmpeg -i render/clifford%1d.svg -vf format=yuv420p clifford_torus.gif
```
The resulting animation is now saved as [clifford_torus.gif](clifford_torus.gif):

<p style="text-align: center;">
    <img title="The Clifford torus" width=500 src="clifford_torus.gif" alt="Not Found";">
</p>

Thus we have recreated the [gif](https://commons.wikimedia.org/wiki/File:Clifford-torus.gif) for the Clifford torus on wikipedia.